In [1]:
import pdfplumber
import pandas as pd
from sqlalchemy import create_engine

In [2]:
# Configs
PDF_FILE = "detailed_invoice.pdf"
DB_USER = "root"
DB_PASSWORD = ""
DB_HOST = "localhost"
DB_NAME = "etl_pdf"
TABLE_NAME = "invoice_data"


In [3]:
def extract_from_pdf(path):
    print(f"📥 Extracting from {path}")
    dataframes = []
    with pdfplumber.open(path) as pdf:
        for page in pdf.pages:
            table = page.extract_table()
            if table:
                df = pd.DataFrame(table[1:], columns=table[0])
                dataframes.append(df)
    return pd.concat(dataframes)

In [4]:
# Transform
def transform_data(df):
    df.columns = [col.strip().lower().replace(" ", "_") for col in df.columns]
    df.fillna("N/A", inplace=True)
    return df

In [5]:
# Load
def load_to_mysql(df):
    engine = create_engine(f"mysql+pymysql://{DB_USER}:{DB_PASSWORD}@{DB_HOST}/{DB_NAME}")
    df.to_sql(TABLE_NAME, con=engine, if_exists='replace', index=False)
    print("✅ Loaded into MySQL table!")

In [6]:
# Run ETL
if __name__ == "__main__":
    raw = extract_from_pdf(PDF_FILE)
    clean = transform_data(raw)
    load_to_mysql(clean)
    print("🎉 ETL Completed")

📥 Extracting from detailed_invoice.pdf
✅ Loaded into MySQL table!
🎉 ETL Completed
